# RetailGenius Customer Churn Prediction

## EPITA - AI Project Methodology 2025-2026

This notebook provides an end-to-end exploration and implementation of the customer churn prediction project.(No need to run since we have ran Makefile)

### Table of Contents
1. [Setup and Imports](#1.-Setup-and-Imports)
2. [Data Loading and Exploration](#2.-Data-Loading-and-Exploration)
3. [Data Preparation](#3.-Data-Preparation)
4. [Feature Engineering](#4.-Feature-Engineering)
5. [Model Training with MLflow](#5.-Model-Training-with-MLflow)
6. [Model Evaluation](#6.-Model-Evaluation)
7. [SHAP Analysis (Explainable AI)](#7.-SHAP-Analysis)
8. [Inference](#8.-Inference)

## 1. Setup and Imports

In [1]:
# Standard library imports
import os
import sys
import warnings
from pathlib import Path

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve
)
import xgboost as xgb
import lightgbm as lgb

# MLflow
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import mlflow.lightgbm

# SHAP
import shap

# Utilities
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("All imports successful!")
print(f"Python version: {sys.version}")
print(f"MLflow version: {mlflow.__version__}")
print(f"SHAP version: {shap.__version__}")

/Users/ebotfabien/Desktop/AI_Project_Methodology/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


All imports successful!
Python version: 3.9.6 (default, Feb  3 2024, 15:58:27) 
[Clang 15.0.0 (clang-1500.3.9.4)]
MLflow version: 3.1.4
SHAP version: 0.49.1


## 2. Data Loading and Exploration

In [2]:
# Load the dataset

DATA_PATH = '../data/raw/E Commerce Dataset.xlsx'

# Try different file formats
try:
    df = pd.read_excel(DATA_PATH)
except:
    try:
        df = pd.read_csv('../data/raw/ecommerce_churn.csv')
    except:
        print("Please place your dataset in data/raw/ directory")
        raise

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

Dataset shape: (21, 4)

Columns: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']


In [3]:
# Display first few rows
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,Data,Variable,Discerption
1,NaN,E Comm,CustomerID,Unique customer ID
2,NaN,E Comm,Churn,Churn Flag
3,NaN,E Comm,Tenure,Tenure of customer in organization
4,NaN,E Comm,PreferredLoginDevice,Preferred login device of customer


In [4]:
# Data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  0 non-null      float64
 1   Unnamed: 1  21 non-null     object 
 2   Unnamed: 2  21 non-null     object 
 3   Unnamed: 3  21 non-null     object 
dtypes: float64(1), object(3)
memory usage: 800.0+ bytes


In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Count plot
sns.countplot(data=df, x='Churn', ax=axes[0], palette='Set2')
axes[0].set_title('Churn Distribution (Count)')

# Pie chart
churn_counts = df['Churn'].value_counts()
axes[1].pie(churn_counts.values, labels=['No Churn', 'Churn'], 
            autopct='%1.1f%%', colors=['#66b3ff', '#ff9999'])
axes[1].set_title('Churn Distribution (Percentage)')

plt.tight_layout()
plt.savefig('../reports/figures/churn_distribution.png', dpi=300)
plt.show()

## 3. Data Preparation

In [ ]:
# Define feature categories
NUMERICAL_FEATURES = [
    'Tenure', 'WarehouseToHome', 'HourSpendOnApp',
    'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress',
    'Complain', 'OrderAmountHikeFromlastYear', 'CouponUsed',
    'OrderCount', 'DaySinceLastOrder', 'CashbackAmount'
]

CATEGORICAL_FEATURES = [
    'PreferredLoginDevice', 'CityTier', 'PreferredPaymentMode',
    'Gender', 'PreferedOrderCat', 'MaritalStatus'
]

TARGET = 'Churn'
ID_COLUMN = 'CustomerID'

In [ ]:
# Create a copy for processing
df_clean = df.copy()

# Remove duplicates
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
print(f"Removed {initial_rows - len(df_clean)} duplicate rows")

# Handle missing values in numerical columns
for col in NUMERICAL_FEATURES:
    if col in df_clean.columns:
        missing = df_clean[col].isnull().sum()
        if missing > 0:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(f"Filled {missing} missing values in '{col}' with median: {median_val}")

# Handle missing values in categorical columns
for col in CATEGORICAL_FEATURES:
    if col in df_clean.columns:
        missing = df_clean[col].isnull().sum()
        if missing > 0:
            mode_val = df_clean[col].mode()[0]
            df_clean[col] = df_clean[col].fillna(mode_val)
            print(f"Filled {missing} missing values in '{col}' with mode: {mode_val}")

In [ ]:
# Save cleaned data
df_clean.to_csv('../data/interim/cleaned_data.csv', index=False)
print(f"Saved cleaned data: {df_clean.shape}")

## 4. Feature Engineering

In [ ]:
# Create derived features
df_features = df_clean.copy()

# Average cashback per order
df_features['AvgCashbackPerOrder'] = df_features['CashbackAmount'] / df_features['OrderCount'].replace(0, 1)

# Engagement score
df_features['EngagementScore'] = df_features['HourSpendOnApp'] * df_features['NumberOfDeviceRegistered']

# Activity level
df_features['ActivityLevel'] = df_features['OrderCount'] / df_features['DaySinceLastOrder'].replace(0, 1)

# Coupon usage rate
df_features['CouponUsageRate'] = df_features['CouponUsed'] / df_features['OrderCount'].replace(0, 1)

print("Created derived features:")
print("- AvgCashbackPerOrder")
print("- EngagementScore")
print("- ActivityLevel")
print("- CouponUsageRate")

In [ ]:
# Encode categorical variables
encoders = {}

for col in CATEGORICAL_FEATURES:
    if col in df_features.columns:
        encoder = LabelEncoder()
        df_features[col] = df_features[col].astype(str)
        df_features[col] = encoder.fit_transform(df_features[col])
        encoders[col] = encoder
        print(f"Encoded '{col}' with {len(encoder.classes_)} classes")

# Save encoders
joblib.dump(encoders, '../models/encoders.joblib')

In [ ]:
# Prepare features and target
feature_cols = [col for col in df_features.columns if col not in [TARGET, ID_COLUMN]]
X = df_features[feature_cols]
y = df_features[TARGET]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {feature_cols}")

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nChurn rate - Train: {y_train.mean():.2%}, Test: {y_test.mean():.2%}")

In [ ]:
# Scale numerical features
scaler = StandardScaler()

# Get all numerical columns (original + derived)
numerical_cols = NUMERICAL_FEATURES + ['AvgCashbackPerOrder', 'EngagementScore', 'ActivityLevel', 'CouponUsageRate']
numerical_cols = [col for col in numerical_cols if col in X_train.columns]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Save scaler
joblib.dump(scaler, '../models/scaler.joblib')

print("Features scaled successfully!")

In [ ]:
# Save processed data
X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

# Save feature names
with open('../data/processed/feature_names.txt', 'w') as f:
    f.write('\n'.join(X_train_scaled.columns.tolist()))

print("Saved processed data to data/processed/")

## 5. Model Training with MLflow

In [ ]:
# Set up MLflow
mlflow.set_tracking_uri('mlruns')
mlflow.set_experiment('RetailGenius_Churn_Prediction')

print("MLflow experiment set up successfully!")

In [ ]:
def evaluate_model(model, X_test, y_test):
    """Evaluate model and return metrics."""
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    return metrics, y_pred, y_pred_proba

In [ ]:
# Model configurations
models = {
    'random_forest': {
        'model': RandomForestClassifier(
            n_estimators=100, max_depth=10, min_samples_split=5,
            min_samples_leaf=2, random_state=RANDOM_STATE, n_jobs=-1
        ),
        'params': {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2}
    },
    'xgboost': {
        'model': xgb.XGBClassifier(
            n_estimators=100, max_depth=6, learning_rate=0.1,
            subsample=0.8, colsample_bytree=0.8, random_state=RANDOM_STATE,
            use_label_encoder=False, eval_metric='logloss'
        ),
        'params': {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 0.8}
    },
    'lightgbm': {
        'model': lgb.LGBMClassifier(
            n_estimators=100, max_depth=6, learning_rate=0.1,
            num_leaves=31, random_state=RANDOM_STATE, verbose=-1
        ),
        'params': {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'num_leaves': 31}
    }
}

In [ ]:
# Train all models with MLflow tracking
results = {}
best_model = None
best_score = 0
best_model_name = None

for model_name, model_config in models.items():
    print(f"\n{'='*50}")
    print(f"Training: {model_name}")
    print(f"{'='*50}")
    
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_params(model_config['params'])
        
        # Train model
        model = model_config['model']
        model.fit(X_train_scaled, y_train)
        
        # Evaluate
        metrics, y_pred, y_pred_proba = evaluate_model(model, X_test_scaled, y_test)
        
        # Log metrics
        for metric_name, metric_value in metrics.items():
            mlflow.log_metric(metric_name, metric_value)
            print(f"  {metric_name}: {metric_value:.4f}")
        
        # Log model
        if model_name == 'random_forest':
            mlflow.sklearn.log_model(model, 'model')
        elif model_name == 'xgboost':
            mlflow.xgboost.log_model(model, 'model')
        elif model_name == 'lightgbm':
            mlflow.lightgbm.log_model(model, 'model')
        
        # Save model locally
        joblib.dump(model, f'../models/{model_name}_model.joblib')
        
        # Store results
        results[model_name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        # Track best model
        if metrics['f1_score'] > best_score:
            best_score = metrics['f1_score']
            best_model = model
            best_model_name = model_name

print(f"\n\nBest Model: {best_model_name} (F1 Score: {best_score:.4f})")

In [ ]:
# Save best model
joblib.dump(best_model, '../models/best_model.joblib')
print(f"Best model saved to models/best_model.joblib")

## 6. Model Evaluation

In [ ]:
# Confusion matrix for best model
best_results = results[best_model_name]
cm = confusion_matrix(y_test, best_results['predictions'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.savefig('../reports/figures/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ROC curves for all models
plt.figure(figsize=(10, 8))

for model_name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['probabilities'])
    auc_score = result['metrics']['roc_auc']
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.savefig('../reports/figures/roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature importance
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X_train_scaled.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=True).tail(15)
    
    plt.figure(figsize=(10, 8))
    plt.barh(importance_df['feature'], importance_df['importance'], color='steelblue')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title(f'Feature Importance - {best_model_name}')
    plt.savefig('../reports/figures/feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()

## 7. SHAP Analysis (Explainable AI)

### Part 3 of the Graded Project

In [ ]:
# Initialize SHAP
shap.initjs()

# Create TreeExplainer
print("Creating SHAP TreeExplainer...")
X_background = X_train_scaled.sample(n=100, random_state=RANDOM_STATE)
explainer = shap.TreeExplainer(best_model, X_background)
print("Explainer created!")

In [ ]:
# Compute SHAP values
print("Computing SHAP values...")
X_sample = X_test_scaled.sample(n=min(500, len(X_test_scaled)), random_state=RANDOM_STATE)
shap_values = explainer(X_sample)
print(f"SHAP values computed for {len(X_sample)} samples!")

In [ ]:
# Waterfall plot for a specific sample
sample_idx = 0

plt.figure(figsize=(12, 8))
if len(shap_values.shape) == 3:
    shap.plots.waterfall(shap_values[sample_idx, :, 1], show=False)
else:
    shap.plots.waterfall(shap_values[sample_idx], show=False)
plt.title(f'SHAP Waterfall Plot - Sample {sample_idx}')
plt.tight_layout()
plt.savefig(f'../reports/figures/shap_waterfall_sample_{sample_idx}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Summary plot (dot)
plt.figure(figsize=(12, 8))
if len(shap_values.shape) == 3:
    shap.summary_plot(shap_values.values[:, :, 1], X_sample, show=False)
else:
    shap.summary_plot(shap_values.values, X_sample, show=False)
plt.title('SHAP Summary Plot')
plt.tight_layout()
plt.savefig('../reports/figures/shap_summary_dot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Mean SHAP values (bar plot)
plt.figure(figsize=(12, 8))
if len(shap_values.shape) == 3:
    shap.summary_plot(shap_values.values[:, :, 1], X_sample, plot_type='bar', show=False)
else:
    shap.summary_plot(shap_values.values, X_sample, plot_type='bar', show=False)
plt.title('Mean Absolute SHAP Values')
plt.tight_layout()
plt.savefig('../reports/figures/shap_mean_values.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Beeswarm plot
plt.figure(figsize=(12, 8))
if len(shap_values.shape) == 3:
    shap.plots.beeswarm(shap_values[:, :, 1], show=False)
else:
    shap.plots.beeswarm(shap_values, show=False)
plt.title('SHAP Beeswarm Plot')
plt.tight_layout()
plt.savefig('../reports/figures/shap_beeswarm.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Force plot for a specific sample
sample_idx = 0

if hasattr(explainer, 'expected_value'):
    if isinstance(explainer.expected_value, np.ndarray):
        expected_value = explainer.expected_value[1]
    else:
        expected_value = explainer.expected_value
else:
    expected_value = shap_values.base_values[sample_idx]

if len(shap_values.shape) == 3:
    sample_shap = shap_values.values[sample_idx, :, 1]
else:
    sample_shap = shap_values.values[sample_idx]

shap.force_plot(expected_value, sample_shap, X_sample.iloc[sample_idx], matplotlib=True, show=False)
plt.title(f'SHAP Force Plot - Sample {sample_idx}')
plt.savefig(f'../reports/figures/shap_force_sample_{sample_idx}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Dependence plots for top features
if len(shap_values.shape) == 3:
    mean_abs_shap = np.abs(shap_values.values[:, :, 1]).mean(axis=0)
else:
    mean_abs_shap = np.abs(shap_values.values).mean(axis=0)

top_features_idx = np.argsort(mean_abs_shap)[-3:][::-1]
top_features = [X_sample.columns[i] for i in top_features_idx]

for feature in top_features:
    plt.figure(figsize=(10, 6))
    if len(shap_values.shape) == 3:
        shap.dependence_plot(feature, shap_values.values[:, :, 1], X_sample, show=False)
    else:
        shap.dependence_plot(feature, shap_values.values, X_sample, show=False)
    plt.title(f'SHAP Dependence Plot - {feature}')
    plt.tight_layout()
    plt.savefig(f'../reports/figures/shap_dependence_{feature}.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Save SHAP feature importance
feature_importance = pd.DataFrame({
    'feature': X_sample.columns,
    'mean_abs_shap': mean_abs_shap
}).sort_values('mean_abs_shap', ascending=False)

feature_importance.to_csv('../reports/figures/shap_feature_importance.csv', index=False)
print("\nTop 10 Features by SHAP Importance:")
print(feature_importance.head(10))

## 8. Inference

In [ ]:
# Load best model for inference
model = joblib.load('../models/best_model.joblib')

# Make predictions
predictions = model.predict(X_test_scaled)
probabilities = model.predict_proba(X_test_scaled)[:, 1]

# Create results DataFrame
inference_results = pd.DataFrame({
    'prediction': predictions,
    'churn_probability': probabilities,
    'risk_level': pd.cut(probabilities, bins=[0, 0.3, 0.6, 1.0], labels=['Low', 'Medium', 'High'])
})

# Save predictions
inference_results.to_csv('../data/processed/predictions.csv', index=False)

print("\nInference Summary:")
print(f"Total samples: {len(inference_results)}")
print(f"Predicted churners: {(predictions == 1).sum()}")
print(f"Predicted non-churners: {(predictions == 0).sum()}")
print(f"\nRisk Level Distribution:")
print(inference_results['risk_level'].value_counts())

## Summary

This notebook demonstrates:

1. **Data Preparation**: Loading, cleaning, and preprocessing the E-Commerce churn dataset
2. **Feature Engineering**: Creating derived features and encoding categorical variables
3. **Model Training**: Training Random Forest, XGBoost, and LightGBM with MLflow tracking
4. **Model Evaluation**: Confusion matrix, ROC curves, and feature importance
5. **SHAP Analysis**: Comprehensive explainability with waterfall, force, summary, beeswarm, and dependence plots
6. **Inference**: Making predictions and risk assessment

### Next Steps
- Run `mlflow ui` to view experiment results
- Deploy the model using `mlflow models serve`
- Check the `reports/figures/` directory for all generated visualizations